In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from skforecast.exceptions import MissingValuesWarning
from skforecast.recursive import ForecasterEquivalentDate


In [3]:
y = pd.Series(
    np.arange(51), 
    index=pd.date_range(start='2000-02-25', periods=51, freq='D')
)


In [ ]:
y_pred = [
                y.loc[(y.index - 1)[1:]]
                for n_off in range(1, 1 + 1)
            ]
y_pred

TypeError: Addition/subtraction of integers and integer-arrays with DatetimeArray is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [5]:
y = pd.Series(
    np.arange(51), 
    index=pd.date_range(start='2000-02-25', periods=51, freq='D')
)

forecaster = ForecasterEquivalentDate(
                    offset    = 1,
                    n_offsets = 1,
                    agg_func  = np.mean
                )
forecaster.fit(y=y)
forecaster.window_size

1

In [11]:
_ = [np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])]
np.vstack(_).shape

(1, 50)

In [8]:
forecaster._binning_in_sample_residuals(y, store_in_sample_residuals=True)
forecaster.in_sample_residuals_

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [24]:
forecaster.last_window_[-6:]

2000-04-10    45
2000-04-11    46
2000-04-12    47
2000-04-13    48
2000-04-14    49
2000-04-15    50
Freq: D, dtype: int64

In [25]:
equivalent_indexes = np.tile(
                            np.arange(-forecaster.offset, 0),
                            int(np.ceil(10 / forecaster.offset))
                        )
print(equivalent_indexes)
equivalent_indexes = equivalent_indexes[:10]
print(equivalent_indexes)

[-3 -2 -1 -3 -2 -1 -3 -2 -1 -3 -2 -1]
[-3 -2 -1 -3 -2 -1 -3 -2 -1 -3]


In [26]:
equivalent_indexes = [
    equivalent_indexes - n * forecaster.offset 
    for n in np.arange(forecaster.n_offsets)
]
print(equivalent_indexes)
equivalent_indexes = np.vstack(equivalent_indexes)
print(equivalent_indexes)
equivalent_values = forecaster.last_window_.to_numpy()[equivalent_indexes]
print(equivalent_values)

[array([-3, -2, -1, -3, -2, -1, -3, -2, -1, -3]), array([-6, -5, -4, -6, -5, -4, -6, -5, -4, -6])]
[[-3 -2 -1 -3 -2 -1 -3 -2 -1 -3]
 [-6 -5 -4 -6 -5 -4 -6 -5 -4 -6]]
[[48 49 50 48 49 50 48 49 50 48]
 [45 46 47 45 46 47 45 46 47 45]]


In [12]:
forecaster.predict(steps=10, last_window=None)

2000-04-16    48
2000-04-17    49
2000-04-18    50
2000-04-19    48
2000-04-20    49
2000-04-21    50
2000-04-22    48
2000-04-23    49
2000-04-24    50
2000-04-25    48
Freq: D, Name: pred, dtype: int64

In [28]:
y.head(3)

2000-02-25    0
2000-02-26    1
2000-02-27    2
Freq: D, dtype: int64

In [121]:
np.random.seed(123)
y_rnd = pd.Series(
    np.random.randint(0, 100, 51), 
    index=pd.date_range(start='2000-02-25', periods=51, freq='D')
)

In [122]:
forecaster = ForecasterEquivalentDate(
                    offset    = 3,
                    n_offsets = 2,
                    agg_func  = np.mean
                )
forecaster.fit(y=y_rnd, store_in_sample_residuals=True)
forecaster.window_size

6

In [123]:
forecaster.in_sample_residuals_

array([ 44.5,   9.5,  18.5,  -4.5, -17. , -44.5, -20.5,  11. , -39. ,
        36.5,  -6.5,   7.5,  31.5,  -7. ,  37.5, -40.5, -24. ,  15. ,
       -70.5,  16.5, -28.5,  40.5,  14.5,  -6. ,  27. , -36. , -36. ,
        35.5,  26. ,  25. ,  17. ,  15. ,  64.5, -71. , -54.5,  24. ,
        14. , -19.5, -92.5,  17.5,  27. ,  25. , -38.5,   6. , -29. ])

In [124]:
forecaster.in_sample_residuals_by_bin_

{0: array([ 7.5, 37.5, 40.5, 25. , 64.5]),
 1: array([ 44.5,  27. ,  27. , -29. ]),
 2: array([ -4.5,  36.5, -36. ,  17.5,  25. ]),
 3: array([ 15. ,  -6. , -19.5,   6. ]),
 4: array([-39. ,  31.5,  35.5,  14. ]),
 5: array([-20.5,  16.5, -28.5,  14.5,  26. ]),
 6: array([-70.5,  15. ,  24. , -38.5]),
 7: array([ 18.5, -44.5, -24. , -36. ,  17. ]),
 8: array([ 9.5, 11. , -6.5, -7. ]),
 9: array([-17. , -40.5, -71. , -54.5, -92.5])}

In [105]:
forecaster.binner_intervals_

{0: (1.5, 5.9),
 1: (5.9, 10.3),
 2: (10.3, 14.7),
 3: (14.7, 19.1),
 4: (19.1, 23.5),
 5: (23.5, 27.9),
 6: (27.9, 32.3),
 7: (32.3, 36.7),
 8: (36.7, 41.1),
 9: (41.1, 45.5)}

In [88]:
forecaster.offset

3

In [126]:
y = pd.Series(
    np.arange(51), 
    index=pd.date_range(start='2000-02-25', periods=51, freq='D')
)

forecaster = ForecasterEquivalentDate(
                    offset    = pd.offsets.MonthEnd(1),
                    n_offsets = 2,
                    agg_func  = np.mean
                )
print(forecaster.window_size)
forecaster.fit(y=y_rnd, store_in_sample_residuals=True)
forecaster.window_size

<2 * MonthEnds>


46

In [127]:
forecaster.in_sample_residuals_

array([ -3.5,   7.5, -33.5,  -9.5, -57.5])

In [129]:
forecaster.binner_intervals_

{0: (67.5, 67.5),
 1: (67.5, 67.5),
 2: (67.5, 67.5),
 3: (67.5, 67.5),
 4: (67.5, 67.5),
 5: (67.5, 67.5),
 6: (67.5, 67.5),
 7: (67.5, 67.5),
 8: (67.5, 67.5),
 9: (67.5, 67.5)}

In [128]:
forecaster.in_sample_residuals_by_bin_

{9: array([ -3.5,   7.5, -33.5,  -9.5, -57.5])}

In [26]:
from pandas.tseries.offsets import DateOffset

y = pd.Series(
        data = np.array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
                         0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
                         0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
                         0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                         0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
                         0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
                         0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
                         0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
                         0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
                         0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]
            ),
        index=pd.date_range(start='2000-01-01', periods=50, freq='D'),
        name = 'y'
    )

In [24]:
y = pd.Series(
    np.arange(51), 
    index=pd.date_range(start='2000-02-25', periods=51, freq='D')
)

forecaster = ForecasterEquivalentDate(
                    offset    = pd.offsets.MonthEnd(1),
                    n_offsets = 2,
                    agg_func  = np.mean
                )
forecaster.fit(y=y, store_in_sample_residuals=True)
forecaster.in_sample_residuals_by_bin_

{9: array([26.5, 27.5, 28.5, 29.5, 30.5])}

In [ ]:

last_window = pd.Series(
    np.arange(50), 
    index=pd.date_range(start='2000-02-10', periods=50, freq='D')
)

warn_msg = re.escape(
    "Steps: ['2000-03-31', '2000-04-01', '2000-04-02'] "
    "are calculated with less than 2 `n_offsets`. "
    "To avoid this, increase the `last_window` size or decrease "
    "the number of `n_offsets`. The current configuration requires " 
    "a total offset of <2 * MonthEnds>."
)
with pytest.warns(MissingValuesWarning, match = warn_msg):
    predictions = forecaster.predict_interval(steps=3, last_window=last_window)

predictions

In [20]:
predictions.to_numpy()

array([[ 0.49422539,  0.05874109,  0.92970969],
       [ 0.332763  , -0.18714832,  0.85267431],
       [ 0.58050578,  0.23107803,  0.92993352],
       [ 0.52551824,  0.09003394,  0.96100255],
       [ 0.57236106,  0.22293331,  0.92178881],
       [ 0.49422539,  0.05874109,  0.92970969],
       [ 0.332763  , -0.18714832,  0.85267431]])

In [6]:
forecaster.binner_intervals_

{0: (0.192909495, 0.41830825),
 1: (0.41830825, 0.5539681),
 2: (0.5539681, 0.850116585)}

In [5]:
forecaster.predict_interval(steps=5)

,pred,lower_bound,upper_bound
2000-02-20,0.501260,0.038421,0.964098
2000-02-21,0.799227,0.315389,1.283066
2000-02-22,0.501260,0.038421,0.964098
2000-02-23,0.799227,0.315389,1.283066
2000-02-24,0.501260,0.038421,0.964098


In [27]:
y_monthly = y.copy()
y_monthly.index = pd.date_range(start='1990-01-01', periods=50, freq='MS')
last_window = pd.Series(
    data = np.array([0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                        0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453]),
    index = pd.date_range(start='2000-08-05', periods=10, freq='MS'),
    name = 'y'
)

forecaster = ForecasterEquivalentDate(
                    offset    = pd.DateOffset(months=2),
                    n_offsets = 2,
                    agg_func  = np.mean
                )
forecaster.fit(y=y_monthly, store_in_sample_residuals=True)
forecaster.out_sample_residuals_ = forecaster.in_sample_residuals_
forecaster.out_sample_residuals_by_bin_ = forecaster.in_sample_residuals_by_bin_
predictions = forecaster.predict_interval(
    steps=7, last_window=last_window, use_in_sample_residuals=False, use_binned_residuals=False
)
predictions

,pred,lower_bound,upper_bound
2001-07-01,0.501260,0.002911,0.999609
2001-08-01,0.799227,0.300878,1.297576
2001-09-01,0.501260,0.002911,0.999609
2001-10-01,0.799227,0.300878,1.297576
2001-11-01,0.501260,0.002911,0.999609
2001-12-01,0.799227,0.300878,1.297576
2002-01-01,0.501260,0.002911,0.999609


In [30]:
predictions.to_numpy()

array([[ 0.42058876,  0.02043732,  0.8207402 ],
       [ 0.87984916,  0.47969772,  1.2800006 ],
       [ 0.5973077 ,  0.19715626,  0.99745913],
       [ 0.49243547,  0.09228403,  0.89258691],
       [ 0.80475637,  0.40460493,  1.20490781],
       [ 0.31755176, -0.08259968,  0.7177032 ],
       [ 0.46509001,  0.06493857,  0.86524144]])

In [14]:
forecaster = ForecasterEquivalentDate(
                     offset    = 7,
                     n_offsets = 2,
                     agg_func  = np.mean
                 )
forecaster.fit(y=y, store_in_sample_residuals=True)
predictions = forecaster.predict_interval(steps=7, use_binned_residuals=False)
predictions

ValueError: The length of `y` (7), must be greater than or equal to the window size (14). This is because  the offset (7) is larger than the available data. Try to decrease the size of the offset (7), the number of `n_offsets` (2) or increase the size of `y`.

In [16]:
y = pd.Series(
    np.arange(7), 
    index=pd.date_range(start='2000-02-25', periods=7, freq='D')
)

forecaster = ForecasterEquivalentDate(
                    offset    = pd.offsets.MonthEnd(1),
                    n_offsets = 1
                )
forecaster._probabilistic_mode = "binned"
print(forecaster.window_size)
forecaster.fit(y=y, store_in_sample_residuals=True)

<MonthEnd>
2


KeyError: "[Timestamp('2000-01-31 00:00:00')] not in index"

In [23]:
y = pd.Series(
    np.arange(7), 
    index=pd.date_range(start='2000-02-25', periods=7, freq='D')
)
y

2000-02-25    0
2000-02-26    1
2000-02-27    2
2000-02-28    3
2000-02-29    4
2000-03-01    5
2000-03-02    6
Freq: D, dtype: int64

In [24]:
n_offsets = 1
offset    = pd.offsets.MonthEnd(1)
window_size = 2

for n_off in range(1, n_offsets + 1):
    idx = y.index - offset * n_off
    mask = idx >= y.index[0]
    print(idx)
    print(mask)
    idx = idx[mask]
    print(idx)
    print(mask.sum())
    print(~mask.sum())
    y_pred = y.loc[idx]
    print(y_pred)

DatetimeIndex(['2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-02-29', '2000-02-29'],
              dtype='datetime64[ns]', freq=None)
[False False False False False  True  True]
DatetimeIndex(['2000-02-29', '2000-02-29'], dtype='datetime64[ns]', freq=None)
2
-3
2000-02-29    4
2000-02-29    4
dtype: int64


In [45]:
y = pd.Series(
    np.arange(10), 
    index=pd.date_range(start='2000-01-01', periods=10, freq='D')
)
y = pd.Series(
    np.arange(7), 
    index=pd.date_range(start='2000-02-25', periods=7, freq='D')
)

n_offsets = 3
offset    = pd.offsets.DateOffset(days=2)
window_size = 6

n_offsets = 1
offset    = pd.offsets.MonthEnd(1)
window_size = 2

y_preds = []
for n_off in range(1, n_offsets + 1):
    print(n_off)
    idx = y.index - offset * n_off
    mask = idx >= y.index[0]
    print(idx)
    print(mask)
    idx = idx[mask]
    print(idx)
    y_pred = y.loc[idx]
    y_pred.index = y_pred.index + (offset * n_off)
    y_preds.append(y_pred)
    print(y_pred)
    print("----------")

1
DatetimeIndex(['2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-02-29', '2000-02-29'],
              dtype='datetime64[ns]', freq=None)
[False False False False False  True  True]
DatetimeIndex(['2000-02-29', '2000-02-29'], dtype='datetime64[ns]', freq=None)
2000-03-31    4
2000-03-31    4
dtype: int64
----------


In [46]:
y

2000-02-25    0
2000-02-26    1
2000-02-27    2
2000-02-28    3
2000-02-29    4
2000-03-01    5
2000-03-02    6
Freq: D, dtype: int64

In [44]:
pd.concat(y_preds, axis=1)

,0,1
2000-03-31,4,NaN
2000-03-31,4,NaN


In [28]:
y_pred = [
    y.loc[(y.index - offset * n_off)[window_size:]]
    for n_off in range(1, n_offsets + 1)
]
y_pred

[2000-01-03    2
 2000-01-04    3
 2000-01-05    4
 2000-01-06    5
 2000-01-07    6
 2000-01-08    7
 dtype: int64,
 2000-01-01    0
 2000-01-02    1
 2000-01-03    2
 2000-01-04    3
 2000-01-05    4
 2000-01-06    5
 dtype: int64]

In [ ]:
y = pd.Series(
    np.arange(100), 
    index=pd.date_range(start='2000-01-01', periods=100, freq='D')
)

n_offsets = 3
offset    = pd.offsets.DateOffset(days=2)
window_size = 6

n_offsets = 2
offset    = pd.offsets.MonthEnd(1)
window_size = 2

y_preds = []
for n_off in range(1, n_offsets + 1):
    print(n_off)
    idx = y.index - offset * n_off
    mask = idx >= y.index[0]
    print(idx)
    print(mask)
    idx = idx[mask]
    print(idx)
    y_pred = y.loc[idx]
    y_pred.index = y_pred.index + (offset * n_off)
    y_preds.append(y_pred)
    print(y_pred)
    print("----------")

1
DatetimeIndex(['1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '1999-12-31',
               '1999-12-31', '1999-12-31', '1999-12-31', '2000-01-31',
               '2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
               '2000-01-31', '2000-01-31', '2000-01-31', '2000-01-31',
    

In [51]:
y

2000-01-01     0
2000-01-02     1
2000-01-03     2
2000-01-04     3
2000-01-05     4
              ..
2000-04-05    95
2000-04-06    96
2000-04-07    97
2000-04-08    98
2000-04-09    99
Freq: D, Length: 100, dtype: int64

In [53]:
y_preds[0]

2000-02-29    30
2000-02-29    30
2000-02-29    30
2000-02-29    30
2000-02-29    30
              ..
2000-04-30    90
2000-04-30    90
2000-04-30    90
2000-04-30    90
2000-04-30    90
Length: 69, dtype: int64

In [55]:
pd.concat(y_preds, axis=1)

,0,1
2000-02-01,30,NaN
2000-02-02,30,NaN
2000-02-03,30,NaN
2000-02-04,30,NaN
2000-02-05,30,NaN
...,...,...
2000-04-05,90,59.0
2000-04-06,90,59.0
2000-04-07,90,59.0
2000-04-08,90,59.0


In [80]:
y = pd.Series(
    np.arange(100), 
    index=pd.date_range(start='2000-01-01', periods=100, freq='D')
)

n_offsets = 3
offset    = pd.offsets.DateOffset(days=2)
window_size = 6

n_offsets = 2
offset    = pd.offsets.MonthEnd(1)
window_size = 2

y_pred = [
    y.shift(offset * n_off)[window_size:]
    for n_off in range(1, n_offsets + 1)
]

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'pandas._libs.tslibs.offsets.MonthEnd'

In [79]:
y_preds = np.column_stack(y_pred)
y_preds

array([[ 4.,  2.,  0.],
       [ 5.,  3.,  1.],
       [ 6.,  4.,  2.],
       [ 7.,  5.,  3.],
       [ 8.,  6.,  4.],
       [ 9.,  7.,  5.],
       [10.,  8.,  6.],
       [11.,  9.,  7.],
       [12., 10.,  8.],
       [13., 11.,  9.],
       [14., 12., 10.],
       [15., 13., 11.],
       [16., 14., 12.],
       [17., 15., 13.],
       [18., 16., 14.],
       [19., 17., 15.],
       [20., 18., 16.],
       [21., 19., 17.],
       [22., 20., 18.],
       [23., 21., 19.],
       [24., 22., 20.],
       [25., 23., 21.],
       [26., 24., 22.],
       [27., 25., 23.],
       [28., 26., 24.],
       [29., 27., 25.],
       [30., 28., 26.],
       [31., 29., 27.],
       [32., 30., 28.],
       [33., 31., 29.],
       [34., 32., 30.],
       [35., 33., 31.],
       [36., 34., 32.],
       [37., 35., 33.],
       [38., 36., 34.],
       [39., 37., 35.],
       [40., 38., 36.],
       [41., 39., 37.],
       [42., 40., 38.],
       [43., 41., 39.],
       [44., 42., 40.],
       [45., 43.

In [ ]:

y_pred = np.apply_along_axis(
                     np.mean,
                     axis = 1,
                     arr  = y_preds
                 )
y_pred

In [68]:
y

2000-01-01     0
2000-01-02     1
2000-01-03     2
2000-01-04     3
2000-01-05     4
              ..
2000-04-05    95
2000-04-06    96
2000-04-07    97
2000-04-08    98
2000-04-09    99
Freq: D, Length: 100, dtype: int64

In [69]:
pd.concat(y_preds, axis=1).mean(axis=1)

2000-01-03     0.0
2000-01-04     1.0
2000-01-05     1.0
2000-01-06     2.0
2000-01-07     2.0
              ... 
2000-04-05    91.0
2000-04-06    92.0
2000-04-07    93.0
2000-04-08    94.0
2000-04-09    95.0
Freq: D, Length: 98, dtype: float64

In [92]:
y = pd.Series(
        np.arange(7), 
        index=pd.date_range(start='2000-02-25', periods=7, freq='D')
    )

n_offsets = 2
offset    = pd.offsets.MonthEnd(1)
window_size = 2

y_preds = []
for n_off in range(1, n_offsets + 1):
    idx = y.index - offset * n_off
    mask = idx >= y.index[0]
    y_pred = y.loc[idx[mask]]
    print(-mask.sum())
    print(y_pred)
    y_pred.index = y.index[-mask.sum():]
    y_preds.append(y_pred)

-2
2000-02-29    4
2000-02-29    4
dtype: int64
0
Series([], dtype: int64)


ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [98]:
y = pd.Series(
        np.arange(10), 
        index=pd.date_range(start='2000-02-25', periods=10, freq='D')
    )

n_offsets = 2
offset    = pd.offsets.DateOffset(days=2)
window_size = 4

y_preds = []
for n_off in range(1, n_offsets + 1):
    idx = y.index - offset * n_off
    mask = idx >= y.index[0]
    y_pred = y.loc[idx[mask]]
    y_pred.index = y.index[-mask.sum():]
    y_preds.append(y_pred)

In [99]:
y_preds = pd.concat(y_preds, axis=1).to_numpy()
y_preds

array([[ 0., nan],
       [ 1., nan],
       [ 2.,  0.],
       [ 3.,  1.],
       [ 4.,  2.],
       [ 5.,  3.],
       [ 6.,  4.],
       [ 7.,  5.]])

In [104]:
y.to_numpy()[-len(y_preds):]

array([2, 3, 4, 5, 6, 7, 8, 9])

In [102]:
np.apply_along_axis(
                     np.mean,
                     axis = 1,
                     arr  = y_preds
                 )

array([nan, nan,  1.,  2.,  3.,  4.,  5.,  6.])

In [ ]:
y = pd.Series(
    np.random.rand(10), index=pd.date_range(start='1/1/2021', periods=10)
)

In [106]:
y_preds = [
    y.shift(5 * n_off)[10:]
    for n_off in range(1, 2 + 1)
]
y_preds

[Series([], Freq: D, dtype: float64), Series([], Freq: D, dtype: float64)]

In [4]:
y = pd.Series(
    np.arange(51), 
    index=pd.date_range(start='2000-02-25', periods=51, freq='D')
)

forecaster = ForecasterEquivalentDate(
                    offset    = pd.offsets.MonthEnd(1),
                    n_offsets = 2,
                    agg_func  = np.mean
                )
forecaster.fit(y=y, store_in_sample_residuals=True)
print(forecaster.binner_intervals_)
forecaster.in_sample_residuals_by_bin_

{0: (19.5, 19.5), 1: (19.5, 19.5), 2: (19.5, 19.5), 3: (19.5, 19.5), 4: (19.5, 19.5), 5: (19.5, 19.5), 6: (19.5, 19.5), 7: (19.5, 19.5), 8: (19.5, 19.5), 9: (19.5, 19.5)}


{9: array([16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 22.5, 23.5, 24.5, 25.5, 26.5,
        27.5, 28.5, 29.5, 30.5]),
 0: array([23.5, 28.5, 18.5, 29.5, 21.5, 30.5, 17.5, 20.5, 16.5, 24.5, 27.5,
        25.5, 26.5, 19.5, 22.5]),
 1: array([22.5, 24.5, 20.5, 23.5, 25.5, 26.5, 29.5, 17.5, 30.5, 27.5, 28.5,
        21.5, 16.5, 18.5, 19.5]),
 2: array([28.5, 26.5, 17.5, 16.5, 30.5, 22.5, 29.5, 18.5, 27.5, 24.5, 20.5,
        25.5, 21.5, 19.5, 23.5]),
 3: array([27.5, 29.5, 23.5, 20.5, 21.5, 25.5, 24.5, 16.5, 19.5, 28.5, 18.5,
        17.5, 30.5, 22.5, 26.5]),
 4: array([24.5, 25.5, 17.5, 21.5, 29.5, 23.5, 18.5, 19.5, 26.5, 16.5, 27.5,
        20.5, 30.5, 22.5, 28.5]),
 5: array([28.5, 29.5, 25.5, 23.5, 27.5, 24.5, 22.5, 16.5, 20.5, 19.5, 18.5,
        21.5, 30.5, 26.5, 17.5]),
 6: array([19.5, 28.5, 30.5, 22.5, 29.5, 25.5, 16.5, 17.5, 21.5, 18.5, 24.5,
        23.5, 26.5, 27.5, 20.5]),
 7: array([19.5, 28.5, 29.5, 25.5, 20.5, 16.5, 17.5, 21.5, 30.5, 18.5, 26.5,
        27.5, 23.5, 24.5, 22.5]),
 